# RMSE calculation - 6 hourly Wxformer

* This notebook shows how RMSEs were computed for 6 hourly Wxformer outputs.

* The notebook runs with `verif_config_6h.yml` in the same folder.

In [1]:
import os
import sys
import yaml
from glob import glob
from datetime import datetime, timedelta

import numpy as np
import xarray as xr

sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu

### Load config

In [2]:
config_name = os.path.realpath('verif_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

### Model and lead time to verify

In [3]:
model_name = 'wxformer'
lead_range = conf[model_name]['lead_range']
verif_lead_range = conf[model_name]['verif_lead_range']

leads_exist = list(np.arange(lead_range[0], lead_range[-1]+lead_range[0], lead_range[0]))
leads_verif = list(np.arange(verif_lead_range[0], verif_lead_range[-1]+verif_lead_range[0], verif_lead_range[0]))
ind_lead = vu.lead_to_index(leads_exist, leads_verif)

print('Verifying lead times: {}'.format(leads_verif))
print('Verifying lead indices: {}'.format(ind_lead))

Verifying lead times: [6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228, 234, 240]
Verifying lead indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [4]:
verif_ind_start = 0; verif_ind_end = 2*(365+366+365)

path_verif = conf[model_name]['save_loc_verif']+'combined_rmse_{:04d}_{:04d}_{:03d}h_{:03d}h_{}.nc'.format(
    verif_ind_start, verif_ind_end, verif_lead_range[0], verif_lead_range[-1], model_name)

### RMSE compute

In [5]:
# ---------------------------------------------------------------------------------------- #
# ERA5 verif target
filename_ERA5 = sorted(glob(conf['ERA5_ours']['save_loc']))

# pick years
year_range = conf['ERA5_ours']['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_ERA5 = [fn for fn in filename_ERA5 if any(year in fn for year in years_pick)]

# merge yearly ERA5 as one
ds_ERA5 = [vu.get_forward_data(fn) for fn in filename_ERA5]
ds_ERA5_merge = xr.concat(ds_ERA5, dim='time')
    
# Select the specified variables and their levels
variables_levels = conf['ERA5_ours']['verif_variables']

# subset merged ERA5 and unify coord names
ds_ERA5_merge = vu.ds_subset_everything(ds_ERA5_merge, variables_levels)

# ---------------------------------------------------------------------------------------- #
# forecast
filename_OURS = sorted(glob(conf[model_name]['save_loc_gather']+'*.nc'))

# pick years
year_range = conf[model_name]['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_OURS = [fn for fn in filename_OURS if any(year in fn for year in years_pick)]
# filename_OURS = [fn for fn in filename_OURS if '00Z' in fn]

L_max = len(filename_OURS)
print('Total number of initialization times: {}'.format(L_max))
assert verif_ind_end <= L_max, 'verified indices (days) exceeds the max index available'

filename_OURS = filename_OURS[verif_ind_start:verif_ind_end]

# latitude weighting
lat = xr.open_dataset(filename_OURS[0])["latitude"]
w_lat = np.cos(np.deg2rad(lat))
w_lat = w_lat / w_lat.mean()

# ---------------------------------------------------------------------------------------- #
# RMSE compute
verif_results = []

for fn_ours in filename_OURS:
    ds_ours = xr.open_dataset(fn_ours)
    ds_ours = vu.ds_subset_everything(ds_ours, variables_levels)
    ds_ours = ds_ours.isel(time=ind_lead)
    ds_ours = ds_ours.compute()
    
    ds_target = ds_ERA5_merge.sel(time=ds_ours['time']).compute()
    
    # RMSE with latitude-based cosine weighting (check w_lat)
    RMSE = np.sqrt(
        (w_lat * (ds_ours - ds_target)**2).mean(['latitude', 'longitude'])
    )
    
    verif_results.append(RMSE.drop_vars('time'))
    
# Combine verif results
ds_verif = xr.concat(verif_results, dim='days')

# # Save the combined dataset
# print('Save to {}'.format(path_verif))
# # ds_verif.to_netcdf(path_verif)

Total number of initialization times: 2192


In [6]:
ds_verif['Z500']

<xarray.DataArray 'Z500' (days: 3, time: 40)>
array([[ 36.33143382,  46.1632761 ,  53.68281897,  71.7214152 ,
         82.15041427,  94.23102534, 100.55727886, 117.34404839,
        127.42406909, 141.57100761, 156.6374341 , 173.60526788,
        190.6631803 , 216.8301137 , 238.34546995, 260.79984018,
        275.73631621, 290.85935767, 309.32386904, 329.83686752,
        355.97858135, 383.17518298, 412.76392658, 438.58562623,
        469.72747645, 500.31482851, 529.74408607, 554.97038274,
        593.4174915 , 634.36477575, 673.21166837, 708.70545933,
        748.50148451, 802.00090821, 841.45900792, 881.39849109,
        908.53456388, 933.95039434, 945.01211855, 953.13967474],
       [ 34.54688872,  45.98413105,  55.31820775,  64.98109086,
         71.28801229,  89.42445016, 101.59341726, 113.92492529,
        127.07335114, 141.05742209, 156.38263814, 178.09935215,
        195.91868303, 217.32184711, 236.03608952, 255.23866597,
        279.10995316, 305.79635944, 333.03637361, 361.32763706,
        390.24953042, 416.38765081, 444.31459163, 469.74648808,
        497.02888022, 520.97481749, 552.58404267, 582.5096208 ,
        615.0975784 , 641.38007014, 671.01204217, 716.32842719,
        754.88226455, 794.8506663 , 821.97994171, 852.96592046,
        869.53761061, 888.64226989, 896.99910263, 916.9612753 ],
       [ 36.41375701,  42.70884359,  47.30686505,  64.61202452,
         72.99774646,  84.09531234,  92.3222681 , 103.80546394,
        114.28351962, 134.62476378, 150.34651468, 169.13442985,
        184.12227968, 199.50032452, 219.86756481, 245.01474233,
        271.13102444, 298.23283317, 321.8715676 , 342.52964473,
        363.35078755, 384.33040704, 404.05996212, 423.25992348,
        449.08842401, 474.97411637, 501.33893385, 523.96374669,
        552.46269691, 599.01705448, 643.81097749, 689.47990343,
        727.9110775 , 767.24180537, 794.19503338, 815.77906488,
        829.27252467, 848.63103078, 857.91573567, 861.64666062]])
Dimensions without coordinates: days, time